In [1]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2


from torch.utils.data import DataLoader
from torch.nn.functional import binary_cross_entropy
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
import torch
import pandas as pd
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import math



In [2]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader
import cv2
import torchvision.transforms.functional as tx
from sklearn.utils import shuffle
import torch
class AL_Dataset():
    def __init__(self, mode):
        if(mode=="train"):
            self.AD_dir = np.array(os.listdir("D:/Project/data_crop/training_data/AD"))
            self.CN_dir = np.array(os.listdir("D:/Project/data_crop/training_data/CN"))
        else:
            self.AD_dir = np.array(os.listdir("D:/Project/data_crop/validation_data/AD"))
            self.CN_dir = np.array(os.listdir("D:/Project/data_crop/validation_data/CN"))
        self.AD = np.ones((len(self.AD_dir)))
        self.CN = np.zeros((len(self.CN_dir)))
        self.df = np.concatenate((self.AD_dir, self.CN_dir))
        self.label = np.concatenate((self.AD, self.CN))
        self.df, self.label = shuffle(self.df, self.label)
        self.mode = mode
        self.label_df = pd.read_csv("../data/B1_correction_new.csv")

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        if self.mode=="train":
            if self.label[idx]==1:
                image_path = "D:/Project/data_crop/training_data/AD/"+self.df[idx]

            else:
                image_path = "D:/Project/data_crop/training_data/CN/"+self.df[idx]

        else:
            if self.label[idx]==1:
                image_path = "D:/Project/data_crop/validation_data/AD/"+self.df[idx]

            else:
                image_path = "D:/Project/data_crop/validation_data/CN/"+self.df[idx]
#         print(image_path)
        patient_ID = image_path.split("/")[5].split(".")[0]
        index = list(self.label_df["Image Data ID"]).index(patient_ID)
        sex = self.label_df["Sex"][index]
        if sex =="M":
            label = 1
        else:
            label = 0
        # print(patient_ID)
        whole_image = nib.load(image_path)
        whole_image = np.array(whole_image.dataobj)
        image_arr = np.zeros((90, 256, 256, 3))
        for i in range((90)):            
            image = whole_image[i]
            image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)
            image = normalize(image)
            image = image.reshape(256, 256, 1)
            image = np.concatenate((image, image, image), 2)
            image_arr[i] = image
        return image_arr, label
    
def normalize(arr):
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    return (arr - arr_min) / (arr_max - arr_min)

In [3]:

def train(dataloader, model, loss_fn, optimizer, writer, epoch, level):

    model.train()
    total_loss = 0
    total_score = 0
    for batch, (X_arr, y) in enumerate(dataloader):
        X = X_arr[:,level]
        X= torch.moveaxis(X,3,1)
        X = X.float()
        y = y.long()
        X, y = X.to(device), y.to(device)

        X = X.requires_grad_()
        # Compute prediction error
        pred = model(X)
        total_score += (np.sum(y.cpu().detach().numpy()==np.argmax(pred.cpu().detach().numpy(), axis = 1))/y.shape[0])
        loss = loss_fn(pred, y)
        total_loss+=(abs(loss)/y.shape[0])
        loss.requires_grad_()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 

        if batch % 25 == 0:
            loss, current = loss.item(), batch * len(X)
            print("total_loss : ", total_loss)
    writer.add_scalar("Loss/train", total_loss/len(dataloader), epoch)
    writer.add_scalar("Score/train", total_score/len(dataloader), epoch)
    return total_loss, total_score




In [10]:
train_dataset = AL_Dataset("train")
val_dataset = AL_Dataset("valid")
train_dataloader = DataLoader(train_dataset, batch_size = 1, shuffle = True)
valid_dataloader = DataLoader(val_dataset, batch_size = 1, shuffle = True)
device = 'cuda' if torch.cuda.is_available() else "cpu"
weight_map = torch.zeros((2))
weight_map[0] = 1
weight_map[1] = 1
weight_map = weight_map.to(device)





In [19]:
score_arr=[]
for i in range(0,90):
    model = models.resnet34(pretrained=False)
    model.fc = nn.Linear(512, 2)
    model.load_state_dict(torch.load("../data/models_sex/level_"+str(i)+"_best.pth"))
    model.to("cuda")
    total_score = 0
    for batch, (X_arr, y) in enumerate(valid_dataloader):
#         if batch%100==0:
#             print(batch)
        X = X_arr[:,i]
        X= torch.moveaxis(X,3,1)
        X = X.float()
        y = y.long()
        X, y = X.to(device), y.to(device)

        X = X.requires_grad_()
        # Compute prediction error
        pred = model(X)
        total_score += (np.sum(y.cpu().detach().numpy()==np.argmax(pred.cpu().detach().numpy(), axis = 1))/y.shape[0])
#         break
    score_arr.append(total_score/len(valid_dataloader))
    print("level : ",i,", mean score : ", total_score/len(valid_dataloader))
#     break

level :  0 , mean score :  0.571917808219178
level :  1 , mean score :  0.5958904109589042
level :  2 , mean score :  0.6164383561643836
level :  3 , mean score :  0.6095890410958904
level :  4 , mean score :  0.6438356164383562
level :  5 , mean score :  0.660958904109589
level :  6 , mean score :  0.571917808219178
level :  7 , mean score :  0.6746575342465754
level :  8 , mean score :  0.6712328767123288
level :  9 , mean score :  0.6472602739726028
level :  10 , mean score :  0.6575342465753424
level :  11 , mean score :  0.6575342465753424
level :  12 , mean score :  0.6678082191780822
level :  13 , mean score :  0.6986301369863014
level :  14 , mean score :  0.660958904109589
level :  15 , mean score :  0.636986301369863
level :  16 , mean score :  0.6335616438356164
level :  17 , mean score :  0.702054794520548
level :  18 , mean score :  0.6815068493150684
level :  19 , mean score :  0.6575342465753424
level :  20 , mean score :  0.7123287671232876
level :  21 , mean score :  0